In [1]:
import numpy as np
import pandas as pd
import pylab as pl
from matplotlib.dates import MonthLocator, YearLocator
from sklearn.linear_model import LinearRegression
from sklearn.ensemble.forest import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

%matplotlib inline
pd.set_option('display.mpl_style', 'default')

def print_missing_stats(train, test, store_info):
    for data_name, data in {'TRAIN': train, 'TEST': test, 'STORE': store_info}.items():
        print(data_name, ' (overall = %d)' % len(data))
        for attribute in data.columns:
            mask = data[attribute].isnull()
            k = len(data[attribute][mask])
            print('%5d (%2d%%)' % (k, 100*k/len(data)), 'missing values in ', attribute) 
        print()

def load_data(filename_train, filename_test, filename_store, print_missing=False):

    train = pd.read_csv(filename_train, header=0, low_memory=False)
    test = pd.read_csv(filename_test, header=0, low_memory=False)
    store_info = pd.read_csv(filename_store, header=0, low_memory=False)

    train.Date = pd.to_datetime(train.Date)
    test.Date = pd.to_datetime(test.Date)
    
    if print_missing:
        print('BEFORE:')
        print_missing_stats(train, test, store_info)
    
    test.Open = test.Open.fillna(1)

    store_info.CompetitionDistance = store_info.CompetitionDistance.fillna(0)
    store_info.CompetitionOpenSinceMonth = store_info.CompetitionOpenSinceMonth.fillna(0).astype(int)
    store_info.CompetitionOpenSinceYear = store_info.CompetitionOpenSinceYear.fillna(0).astype(int)
    store_info.Promo2SinceWeek = store_info.Promo2SinceWeek.fillna(0).astype(int)
    store_info.Promo2SinceYear = store_info.Promo2SinceYear.fillna(0).astype(int)

    promo_intervals = [np.NaN] + list(store_info.PromoInterval.value_counts().index)
    store_info.PromoInterval = store_info.PromoInterval.map(lambda x: promo_intervals.index(x))
    
    if print_missing:
        print('AFTER:')
        print_missing_stats(train, test, store_info)
    return train, test, store_info

In [2]:
def plot_all_store_sales(train):
    fig, axes = pl.subplots(nrows=7, ncols=1, sharey=True, figsize=(20,100))

    open_df = train[train.Open == 1]
    for day_of_week in range(1, 8):
        custom_df = open_df[open_df.DayOfWeek == day_of_week] 
        gp_store = custom_df.groupby('Store')

        for store, group in gp_store:
            axes[day_of_week - 1].plot(group['Date'], group['Sales'], 'v--')

        gp_date = custom_df.groupby('Date')

        ts_mean = gp_date['Sales'].mean()
        ts_median = gp_date['Sales'].median()
        ts_mean.plot(style='r-', linewidth=5, ax=axes[day_of_week - 1], label='mean')
        ts_median.plot(style='b-', linewidth=5, ax=axes[day_of_week - 1], label='median')


        axes[day_of_week - 1].set_title('Day ' + str(day_of_week) + '. number of stores = ' + str(len(gp_store)))
        axes[day_of_week - 1].legend()
        axes[day_of_week - 1].xaxis.set_major_locator(MonthLocator())
        axes[day_of_week - 1].grid(True)
    pl.savefig('all_stores_and_median.png', format='png')

In [3]:
def construct_label_name(school_holiday, state_holiday, promo_flag, n_stores):
    string_school = 'NO SchoolHoliday. '
    string_state = 'NO StateHoliday. '
    string_promo = 'NO Promo. '
    if school_holiday == 1:
        string_school = string_school[3:]
    if promo_flag:
        string_promo = string_promo[3:]
    if state_holiday != '0':
        string_state = {'a': 'PublicHoliday. ',
                        'b': 'EasterHoliday. ',
                        'c':'Christmas. '}[state_holiday]
    string_stores = '(' + str(n_stores) + ' stores)'
    return string_school + string_state + string_promo + string_stores

def plot_mean_decomposition(train):
    fig, axes = pl.subplots(nrows=7, ncols=1, sharey=True, figsize=(20,100))

    open_df = train[train.Open == 1]
    for day_of_week in range(1, 8):
        day_df = open_df[open_df.DayOfWeek == day_of_week]
        for school_holiday in [0, 1]:
            school_df = day_df[day_df.SchoolHoliday == school_holiday]
            for state_holiday in ['0', 'a', 'b', 'c']:
                state_df = school_df[school_df.StateHoliday == state_holiday]
                for promo_flag in [0, 1]:
                    custom_df = state_df[state_df.Promo == promo_flag]
                    if not custom_df.empty:
                        gp_date = custom_df.groupby('Date')
                        gp_store = custom_df.groupby('Store')

                        ts_mean = gp_date.Sales.mean()
                        axes[day_of_week - 1].plot(ts_mean.index, ts_mean, 'v--', 
                                                   label=construct_label_name(school_holiday, state_holiday, 
                                                                              promo_flag, len(gp_store)))

        custom_df = day_df
        gp_date = custom_df.groupby('Date')
        gp_store = custom_df.groupby('Store')
        ts_mean = gp_date.Sales.mean()
        ts_mean.plot(style='r-', linewidth=1.5, ax=axes[day_of_week - 1],
                     label='mean (' + str(len(gp_store)) + ' stores)')
        axes[day_of_week - 1].set_title('Day ' + str(day_of_week))
        axes[day_of_week - 1].legend()
        axes[day_of_week - 1].xaxis.set_major_locator(MonthLocator())
        axes[day_of_week - 1].grid(True)
    
    
    pl.savefig('median_decomposition.png', format='png')

In [4]:
def get_dates_CV(k):
    if k >= 20:
        k = 19
        print('maximum number of cross-validation folds is 19')
    date_range = pd.date_range('2013-01-01', '2015-07-31')
    for i in range(k):
        temp_date_range = date_range.shift(-i*48)[48*i:]
        train_date_range = temp_date_range[:-48]
        test_date_range = temp_date_range[-48:] 
        yield train_date_range, test_date_range

In [5]:
def merge_with_store(data, store_info):    
    store_features = ['Store', 'StoreType', 'Assortment','CompetitionDistance', 
                      'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                      'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
    data = pd.merge(data, store_info[store_features], on='Store', how='left')
    return data

def get_dummies_values(series, prefix, values):
    new_series = pd.get_dummies(series, prefix=prefix)
    for value in values:
        column_name = prefix + '_' + str(value)
        if column_name not in new_series.columns:
            new_series = new_series.join(pd.DataFrame(np.zeros(len(new_series)), index=new_series.index,
                                                   columns=[column_name]))
    return new_series

In [6]:
def construct_some_features(data):
    data['Day'] = data.Date.map(lambda d: d.day).astype(int)
    data['Month'] = data.Date.map(lambda d: d.month).astype(int)
    data['Week'] = data.Date.map(lambda d: d.week).astype(int)
    data['DayOfYear'] = data.Date.map(lambda d: d.dayofyear).astype(int)
    data['Year'] = data.Date.map(lambda d: d.year)
   
    data['DiffToday'] = ((pd.datetime(2015, 9, 18) - data.Date)
                                     / np.timedelta64(1, 'D')).astype(int)
    data['DiffNewYear'] = data.Date.map(lambda d: (min(d - pd.datetime(d.year, 1, 1),
                                  pd.datetime(d.year + 1, 12, 31) - d) / np.timedelta64(1, 'D')).astype(int))
    data['DiffFoolDay'] = data.Date.map(lambda d: ((d - pd.datetime(d.year, 4, 1))
                                                                           / np.timedelta64(1, 'D')).astype(int))

    data = data.join(get_dummies_values(data.StateHoliday, prefix='StateHoliday', values=['0', 'a', 'b', 'c']))
    data = data.join(get_dummies_values(data.Month, prefix='Month', values=np.arange(1, 13)))
    data = data.join(get_dummies_values(data.StoreType, prefix='StoreType', values=['a', 'b', 'c', 'd']))
    data = data.join(get_dummies_values(data.Assortment, prefix='Assortment', values=['a', 'b', 'c']))
    data = data.join(get_dummies_values(data.DayOfWeek, prefix='DayOfWeek', values=np.arange(1, 8)))
    data = data.join(get_dummies_values(data.CompetitionOpenSinceMonth, prefix='CompetitionOpenSinceMonth',
                                        values=np.arange(1, 13)))
    
    del data['Date'], data['StateHoliday'], data['Month'],
    del data['StoreType'], data['Assortment'], data['DayOfWeek'],
    del data['CompetitionOpenSinceMonth']
    return data

In [7]:
def construct_train(train_data, store_info, train_features):
    train_data = train_data[train_features + ['Sales']]
        
    train_data = merge_with_store(train_data, store_info)
    train_data = construct_some_features(train_data)

    train_data_labels = np.array(train_data.Sales)
    del train_data['Sales']
    
    features = list(train_data.columns)
    return np.array(train_data), train_data_labels, features


def construct_test(data, store_info, train_features):
    data = merge_with_store(data[train_features], store_info)
    data = construct_some_features(data)
    
    features = list(data.columns)
    return np.array(data), features

In [8]:
def main_feature_construction(data, validation, date_ranges=None):
    train, test, store_info = data
    train = train[train.Open == 1]
    
    if validation:
        date_range_train, date_range_test = date_ranges
        train_data = train[train.Date.isin(date_range_train)]
        test_data = train[train.Date.isin(date_range_test)]
    else:
        train_data = train
        test_data = test
    train_features = ['Store', 'DayOfWeek', 'Date', 'Promo', 'SchoolHoliday', 'StateHoliday']
    
    train_data, train_data_labels, features_names_train = construct_train(train_data, store_info, train_features)
    
    if validation:
        test_data_labels = np.asarray(test_data.Sales)
    else:
        test_data_labels = None

    test_data, features_names_test = construct_test(test_data, store_info, train_features)
    
    return train_data, train_data_labels, test_data, test_data_labels, features_names_train

### Predicting

In [9]:
def visualize_feature_importances(clf, features):
    importances = clf.feature_importances_
    std = np.std([tree.feature_importances_ for tree in clf.estimators_], axis=0)
    sorted_indices = np.argsort(importances)[::-1]
    for i, k in enumerate(sorted_indices):
        print('%2d (feature %2d):' % (i, k), features[k], 'Importance = %.5f' % importances[k])
    pl.figure(figsize=(20,10))
    pl.title('Feature Importance')
    pl.bar(range(len(features)), importances[sorted_indices], 
           color='r', yerr=std[sorted_indices], align='center')
    pl.xticks(range(len(features)), sorted_indices)
    pl.xlim([-1, len(features) + 1])
#     pl.savefig('feature_importance.png', format='png')

def compute_RMSPE(test_labels, predicted_labels):
    mask = test_labels.nonzero()
    y = test_labels[mask]
    y_hat = predicted_labels[mask]
    return np.sqrt(np.mean(((y - y_hat)/y)**2))

def visualize_RMSPE(test_labels, predicted_labels, step = 0.05):
    for i in np.arange(0, 1, step):
        pl.figure(figsize=(20,7))
        pl.plot(test_labels[i*len(test_labels):(i+step)*len(test_labels)],
                'b.', label='real')
        pl.plot(predicted_labels[i*len(predicted_labels):(i+step)*len(predicted_labels)],
                'r.', label='predicted')
        pl.legend()

In [10]:
def fit_predict_model(train_data, train_data_labels, test_data,
                      test_data_labels, validation, params=None,
                      print_importances=True, print_rmspe=False):
    if params:
        clf = RandomForestRegressor(n_jobs=-1, **params)
    else:
        clf = RandomForestRegressor(n_estimators=30, n_jobs=-1, min_samples_leaf=2,
                                    max_features=0.464285714286)
    
#     clf = LinearRegression(normalize=True, n_jobs=-1)
    
    clf.fit(train_data, np.log(train_data_labels + 1.))
    if print_importances and hasattr(clf, 'feature_importances_'):
        visualize_feature_importances(clf, features)
    
    test_predicted_labels = np.exp(clf.predict(test_data)) - 1
    if validation: 
        rmspe_score = compute_RMSPE(test_data_labels, test_predicted_labels)
    else:
        rmspe_score = None
        
    return test_predicted_labels, rmspe_score

### script begins

In [11]:
data = load_data('train.csv', 'test.csv', 'store.csv')

In [12]:
# plot_all_store_sales(data[0])

In [ ]:
# plot_mean_decomposition(data[0])

#Main

In [ ]:
validation = True

if validation:
    rmspe_cv = 0. # 58 features -> 0.
#     params = {
#         'n_estimators': range(30, 51, 5),
#         'min_samples_lead': range(1,5),
#         'max_features': ['auto', 'sqrt', 'log2'] + list(range(5, 31, 5))
#     }
    folds = 1
    if folds == 1:
        print_importances = False
    else:
        print_importances = True
    for i, date_ranges in enumerate(get_dates_CV(folds)):
        print('Folding', i+1)
        print('Train -> ', [str(date_ranges[0][0]), str(date_ranges[1][-1])])
        print(' Test -> ', [str(date_ranges[1][0]), str(date_ranges[1][-1])])
        result = main_feature_construction(data, validation, date_ranges=date_ranges)
        train_data, train_data_labels, test_data, test_data_labels, features = result
        test_predicted_labels, rmspe_score = fit_predict_model(train_data, train_data_labels, test_data,
                                                               test_data_labels, validation,
                                                               print_importances=print_importances)
        print('rmspe_score = %1.6f' % rmspe_score)
        rmspe_cv += rmspe_score
    rmspe_cv /= folds
    print('Number of features =', len(features))
    print('Cross-validated RMSPE = %1.6f' % rmspe_cv)
    
else:
    print_importances = True
    result = main_feature_construction(data, validation)
    train_data, train_data_labels, test_data, test_data_labels, features = result
    test_predicted_labels, rmspe_score = fit_predict_model(train_data, train_data_labels, test_data,
                                                           test_data_labels, validation,
                                                           print_importances=print_importances)

if not validation:
    test = data[1][['Id', 'Open']]
    test['Sales'] = test_predicted_labels
    test.ix[test.Open == 0, 'Sales'] = 0
    test[['Id', 'Sales']].to_csv('prediction.csv', index=False)
    print('\nResult was written to prediction.csv')
    del test